In [9]:
!pip3 install dgl
!export DGLBACKEND=tensorflow
!python -m dgl.backend.set_default_backend tensorflow
!pip install torch torchvision torchaudio

/usr/local/Cellar/python@2.7.17/2.7.17_1/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: No module named dgl.backend
     |████████████████████████████████| 108.9 MB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 11.5 MB/s eta 0:00:01


In [26]:
import dgl
import dgl.function as fn
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F

In [12]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [13]:
g = dataset[0]
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}
Edge features
{}


In [16]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [17]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.088 (best 0.088), test acc: 0.080 (best 0.080)
In epoch 5, loss: 1.898, val acc: 0.476 (best 0.476), test acc: 0.517 (best 0.517)
In epoch 10, loss: 1.819, val acc: 0.592 (best 0.592), test acc: 0.627 (best 0.627)
In epoch 15, loss: 1.716, val acc: 0.648 (best 0.648), test acc: 0.675 (best 0.675)
In epoch 20, loss: 1.589, val acc: 0.688 (best 0.688), test acc: 0.711 (best 0.706)
In epoch 25, loss: 1.441, val acc: 0.692 (best 0.692), test acc: 0.722 (best 0.722)
In epoch 30, loss: 1.278, val acc: 0.706 (best 0.706), test acc: 0.731 (best 0.731)
In epoch 35, loss: 1.107, val acc: 0.724 (best 0.724), test acc: 0.735 (best 0.735)
In epoch 40, loss: 0.938, val acc: 0.734 (best 0.734), test acc: 0.745 (best 0.745)
In epoch 45, loss: 0.778, val acc: 0.734 (best 0.736), test acc: 0.754 (best 0.747)
In epoch 50, loss: 0.636, val acc: 0.744 (best 0.744), test acc: 0.757 (best 0.754)
In epoch 55, loss: 0.514, val acc: 0.754 (best 0.754), test acc: 0.769 (best 0

In [19]:
g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]))

# Assign a 3-dimensional node feature vector for each node.
g.ndata['x'] = torch.randn(6, 3)
# Assign a 4-dimensional edge feature vector for each edge.
g.edata['a'] = torch.randn(5, 4)
# Assign a 5x4 node feature matrix for each node.  Node and edge features in DGL can be multi-dimensional.
g.ndata['y'] = torch.randn(6, 5, 4)
newg = dgl.add_reverse_edges(g)
newg.edges()

(tensor([0, 0, 0, 0, 0, 1, 2, 3, 4, 5]),
 tensor([1, 2, 3, 4, 5, 0, 0, 0, 0, 0]))

In [21]:
dgl.save_graphs('graph.dgl', g)
(g,),s = dgl.load_graphs('graph.dgl')
print(g)
print(s)

Graph(num_nodes=6, num_edges=5,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'y': Scheme(shape=(5, 4), dtype=torch.float32)}
      edata_schemes={'a': Scheme(shape=(4,), dtype=torch.float32)})
{}


In [37]:
#Does not work yet
def u_mul_e_udf(edges):
    return {'m' : edges.src['h'] * edges.data['w']}

In [44]:
class WeightedSAGEConv(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h, w):
        with g.local_scope():
            g.ndata['h'] = h
            g.edata['w'] = w
            g.update_all(message_func=fn.u_mul_e('h', 'w', 'm'), reduce_func=fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [45]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = WeightedSAGEConv(in_feats, h_feats)
        self.conv2 = WeightedSAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat, torch.ones(g.num_edges()).to(g.device))
        h = F.relu(h)
        h = self.conv2(g, h, torch.ones(g.num_edges()).to(g.device))
        return h

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]
model = Model(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
In epoch 0, loss: 1.951, val acc: 0.122 (best 0.122), test acc: 0.130 (best 0.130)
In epoch 5, loss: 1.889, val acc: 0.392 (best 0.392), test acc: 0.420 (best 0.420)
In epoch 10, loss: 1.770, val acc: 0.294 (best 0.540), test acc: 0.290 (best 0.526)
In epoch 15, loss: 1.585, val acc: 0.388 (best 0.540), test acc: 0.377 (best 0.526)
In epoch 20, loss: 1.337, val acc: 0.562 (best 0.562), test acc: 0.535 (best 0.535)
In epoch 25, loss: 1.051, val acc: 0.666 (best 0.666), test acc: 0.656 (best 0.656)
In epoch 30, loss: 0.763, val acc: 0.726 (best 0.726), test acc: 0.730 (best 0.730)
In epoch 35, loss: 0.513, val acc: 0.750 (best 0.750), test acc: 0.761 (best 0.761)
In epoch 40, loss: 0.323, val acc: 0.760 (best 0.760), test acc: 0.769 (best 0.769)
In epoch 45, loss: 0.197, val acc: 0.764 (best 0.764), test acc: